In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)))

from keras.applications import Xception
from keras import models
from keras import layers
from keras import optimizers

import numpy as np
import gc

"""
./format_data/train/
./format_data/val/
"""
train_dir = "./format_data/train/"
test_dir = "./format_data/val/"

feature_size = 299
batch_size = 5
middle_feature_size = (10,10,2048)
middle_sample_size = 3000

Using TensorFlow backend.


In [2]:
train_gen = ImageDataGenerator(rescale=1./255,rotation_range=180,horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

train_gener = train_gen.flow_from_directory(train_dir,target_size=(feature_size,feature_size),batch_size=batch_size,shuffle = True,seed = 100,class_mode="categorical")
test_gener = test_gen.flow_from_directory(test_dir,target_size=(feature_size,feature_size),batch_size=batch_size,class_mode="categorical")

for data_batch,label_batch in train_gener:
    print(data_batch.shape)
    print(label_batch.shape)
    break

Found 1886 images belonging to 11 classes.
Found 110 images belonging to 11 classes.
(5, 299, 299, 3)
(5, 11)


In [3]:
X_model = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
X_model.load_weights("./xception_weights_notop.h5")

In [4]:
X_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [5]:
def extract_feature(gener,sample_count = 1000):
    features = np.zeros(shape = (sample_count,middle_feature_size[0],middle_feature_size[1],middle_feature_size[2]))
    labels = np.zeros(shape = (sample_count,11))
    
    i = 0
    for input_batch,label_batch in gener:
        feature_batch = X_model.predict(input_batch)
        features[i * batch_size: (i+1) * batch_size] = feature_batch
        labels[i * batch_size : (i+1) * batch_size] = label_batch
        
        i+=1
        if i*batch_size >= sample_count:
            break
    
    return features,labels

In [6]:
train_features,train_labels = extract_feature(train_gener,middle_sample_size)
val_features,val_labels = extract_feature(test_gener,110)

train_features = np.reshape(train_features,(middle_sample_size,middle_feature_size[0]*middle_feature_size[1]*middle_feature_size[2] ))
val_features = np.reshape(val_features,(110,middle_feature_size[0]*middle_feature_size[1]*middle_feature_size[2] ))


In [4]:
model = models.Sequential()

model.add(layers.Dense(512,activation='relu',input_dim = middle_feature_size[0]*middle_feature_size[1]*middle_feature_size[2]))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11,activation='softmax'))

gc.collect()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               104858112 
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 11)                5643      
Total params: 104,863,755
Trainable params: 104,863,755
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),loss='categorical_crossentropy',metrics=['acc'])

In [9]:
history = model.fit(train_features,train_labels,epochs=5,batch_size=50,validation_data=(val_features,val_labels))

Train on 3000 samples, validate on 110 samples
Epoch 1/5
3000/3000 [==============================] - 122s - loss: 1.5608 - acc: 0.5590 - val_loss: 2.3776 - val_acc: 0.3000
Epoch 2/5
3000/3000 [==============================] - 110s - loss: 1.0536 - acc: 0.6743 - val_loss: 2.3403 - val_acc: 0.3182
Epoch 3/5
3000/3000 [==============================] - 109s - loss: 0.8987 - acc: 0.7120 - val_loss: 1.8473 - val_acc: 0.5182
Epoch 4/5
3000/3000 [==============================] - 111s - loss: 0.7374 - acc: 0.7643 - val_loss: 2.0307 - val_acc: 0.5182
Epoch 5/5
3000/3000 [==============================] - 109s - loss: 0.6360 - acc: 0.7900 - val_loss: 2.2759 - val_acc: 0.4727


In [10]:
model.save('Xception_top.h5')
print('save done')

save done
